### Task - 1

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt


path='Data/Audio/'

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dropout, GlobalMaxPool2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
for (dirpath, dirnames, filenames) in os.walk(path):
    if not dirnames:
        print(dirpath)
        print(len(os.listdir(dirpath)))

Data/Audio/Female\PTDB-TUG
120
Data/Audio/Female\TMIT
205
Data/Audio/Male\PTDB-TUG
120
Data/Audio/Male\TMIT
64
Data/Audio/Noizeus\Babble
30
Data/Audio/Noizeus\Car
30
Data/Audio/Noizeus\NoNoise
30
Data/Audio/Noizeus\Restaurant
30
Data/Audio/Noizeus\Station
30
Data/Audio/Noizeus\Street
30
Data/Audio/Noizeus\Train
30


In [39]:
import os
import pandas as pd
from scipy.io import wavfile
import matplotlib.pyplot as plt

def process_audio_data(data_folder, output_base_directory):
    audio_paths = []  # List to store audio file paths
    class_labels = []  # List to store class labels
    count = 0

    def generate_spectrogram(audio_path, output_directory):
        nonlocal count
        try:
            # Read audio file
            sample_rate, audio_data = wavfile.read(audio_path)

            # Generate spectrogram
            Pxx, freqs, bins, im = plt.specgram(audio_data, NFFT=1024, Fs=8000, noverlap=900)
            plt.axis('off')

            # Create output directory if it doesn't exist
            os.makedirs(output_directory, exist_ok=True)

            # Save the spectrogram image
            output_filename = os.path.join(output_directory, os.path.splitext(os.path.basename(audio_path))[0] + '.png')
            plt.savefig(output_filename, bbox_inches='tight')
            plt.close()  # Close the figure to release resources

        except ValueError as ve:
            print(f"Error during spectrogram computation for {audio_path}: {ve}")

    for root, dirs, files in os.walk(data_folder):
        for file in files:
            audio_path = os.path.join(root, file)

            # Check if the file is a WAV file
            if audio_path.lower().endswith('.wav'):
                # Use the relative path from data_folder as the class label
                class_label = os.path.relpath(root, data_folder)
                audio_paths.append(audio_path)
                class_labels.append(class_label)

                # Generate spectrogram
                output_directory = os.path.join(output_base_directory, class_label)
                generate_spectrogram(audio_path, output_directory)

                count += 1
                print(count, end=',')

    # Create a DataFrame from the collected data
    df = pd.DataFrame({'Path': audio_paths, 'Class': class_labels})
    return df


In [40]:

data_folder = 'Data/Audio/'
output_base_directory = 'spectrograms_/'

df_result = process_audio_data(data_folder, output_base_directory)
df_result.head()

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,Error during spectrogram computation for Data/Audio/Female\TMIT\SI1160.wav: only 1-dimensional arrays can be used
130,131,Error during spectrogram computation for Data/Audio/Female\TMIT\SI1208.wav: only 1-dimensional arrays can be used
132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,Error during spectrogram computation for Data/Audio/Female\TMIT\SI1790.wav: only 1-dimensional arrays can be used
157,158,Error during spectrogram computation for Data/Audio/Female\TMIT\SI1838.wav: only 1-dimensional arrays can be used
159,160,161,162,163,

,Path,Class
0,Data/Audio/Female\PTDB-TUG\mic_F01_sa2.wav,Female\PTDB-TUG
1,Data/Audio/Female\PTDB-TUG\mic_F01_si454.wav,Female\PTDB-TUG
2,Data/Audio/Female\PTDB-TUG\mic_F01_si473.wav,Female\PTDB-TUG
3,Data/Audio/Female\PTDB-TUG\mic_F01_si502.wav,Female\PTDB-TUG
4,Data/Audio/Female\PTDB-TUG\mic_F01_si523.wav,Female\PTDB-TUG


### Task - 2

In [10]:
def load_data(spec_dir):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_generator = datagen.flow_from_directory(
        spec_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode="categorical",
        subset="training",
        shuffle=True
    )

    val_generator = datagen.flow_from_directory(
        spec_dir,
        target_size=(150, 150),
        batch_size=16,
        class_mode="categorical",
        subset="validation",
        shuffle=True
    )

    return train_generator, val_generator

In [11]:
def build_encoder(input_size):
    model = tf.keras.Sequential([
        Conv2D(32, (3, 3), input_shape=input_size, activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(2, 2),
        Dropout(0.2),

        Conv2D(64, (3, 3), activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(2, 2),
        Dropout(0.2),

        Conv2D(128, (3, 3), activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPool2D(2, 2),
        Dropout(0.2),

        GlobalMaxPool2D()
    ])
    return model
def build_model(input_size):
    input_layer = Input(input_size)
    encoder = build_encoder(input_size)(input_layer)
    output_layer = Dense(3, activation='softmax')(encoder)
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

def compile_model(model):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
def train_model(model, train_generator, val_generator):
    checkpoint_filepath = 'model/ckpt/checkpoint.model.keras'
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True
    )

    early = EarlyStopping(
        patience=5,
        min_delta=0.001,
        restore_best_weights=True
    )

    model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=[early])

    return model
def evaluate_model(model, val_generator):
    return model.evaluate(val_generator)

# Load data
spec_dir = "spectrogram/"
train_generator, val_generator = load_data(spec_dir)

# Build model
input_size = train_generator.image_shape
model = build_model(input_size)

# Compile model
compiled_model = compile_model(model)

# Train model
trained_model = train_model(compiled_model, train_generator, val_generator)

# Evaluate model
evaluation_result = evaluate_model(trained_model, val_generator)
print("Evaluation Result:", evaluation_result)

Found 576 images belonging to 3 classes.
Found 143 images belonging to 3 classes.


Epoch 1/20
18/18 [==============================] - 34s 2s/step - loss: 1.0846 - accuracy: 0.4132 - val_loss: 1.0460 - val_accuracy: 0.4545
Epoch 2/20
18/18 [==============================] - 33s 2s/step - loss: 0.9018 - accuracy: 0.5799 - val_loss: 0.6593 - val_accuracy: 0.7273
Epoch 3/20
18/18 [==============================] - 33s 2s/step - loss: 0.6960 - accuracy: 0.6719 - val_loss: 0.6024 - val_accuracy: 0.7413
Epoch 4/20
18/18 [==============================] - 33s 2s/step - loss: 0.6347 - accuracy: 0.6979 - val_loss: 0.5311 - val_accuracy: 0.7413
Epoch 5/20
18/18 [==============================] - 33s 2s/step - loss: 0.6976 - accuracy: 0.6719 - val_loss: 0.9568 - val_accuracy: 0.4615
Epoch 6/20
18/18 [==============================] - 33s 2s/step - loss: 0.7453 - accuracy: 0.6701 - val_loss: 0.6220 - val_accuracy: 0.7413
Epoch 7/20
18/18 [==============================] - 34s 2s/step - loss: 0.6379 - accuracy: 0.6927 - val_loss: 0.5585 - val_accuracy: 0.7483
Epoch 8/20
18/18 [==

In [13]:
model.save('model.h5')


### Task - 3

In [41]:
import os
import pandas as pd
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np

def process_noisy_audio_data(data_folder, output_base_directory):
    audio_paths = []  # List to store audio file paths
    class_labels = []  # List to store class labels
    count = 0

    def generate_noisy_spectrogram(audio_path, output_directory):
        nonlocal count
        try:
            # Read audio file
            sample_rate, audio_data = wavfile.read(audio_path)

            # Add random noise to the audio
            noise = np.random.normal(0, 0.001, audio_data.shape[0])
            noisy_wave = audio_data + noise

            # Generate spectrogram
            Pxx, freqs, bins, im = plt.specgram(noisy_wave, NFFT=1024, Fs=8000, noverlap=900)
            plt.axis('off')

            # Create output directory if it doesn't exist
            os.makedirs(output_directory, exist_ok=True)

            # Save the spectrogram image
            output_filename = os.path.join(output_directory, os.path.splitext(os.path.basename(audio_path))[0] + '.png')
            plt.savefig(output_filename, bbox_inches='tight')
            plt.close()  # Close the figure to release resources

        except ValueError as ve:
            print(f"Error during spectrogram computation for {audio_path}: {ve}")

    for root, dirs, files in os.walk(data_folder):
        for file in files:
            audio_path = os.path.join(root, file)

            # Check if the file is a WAV file
            if audio_path.lower().endswith('.wav'):
                # Use the relative path from data_folder as the class label
                class_label = os.path.relpath(root, data_folder)
                audio_paths.append(audio_path)
                class_labels.append(class_label)

                # Generate noisy spectrogram
                output_directory = os.path.join(output_base_directory, class_label)
                generate_noisy_spectrogram(audio_path, output_directory)

                count += 1
                print(count, end=',')

    # Create a DataFrame from the collected data
    df = pd.DataFrame({'Path': audio_paths, 'Class': class_labels})
    return df


In [42]:
data_folder_noisy = 'Data/Audio/'
output_base_directory_noisy = 'noisy_spectrograms/'

df_result_noisy = process_noisy_audio_data(data_folder_noisy, output_base_directory_noisy)
df_result_noisy.head()

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,Error during spectrogram computation for Data/Audio/Female\TMIT\SI1160.wav: operands could not be broadcast together with shapes (232848,2) (232848,) 
130,131,Error during spectrogram computation for Data/Audio/Female\TMIT\SI1208.wav: operands could not be broadcast together with shapes (136040,2) (136040,) 
132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,Error during spectrogram computation for Data/Audio/Female\TMIT\SI1790.wav: operands could not be broadcast together with shapes (102736,2) (102736,) 
157,158,Error during spectrogra

,Path,Class
0,Data/Audio/Female\PTDB-TUG\mic_F01_sa2.wav,Female\PTDB-TUG
1,Data/Audio/Female\PTDB-TUG\mic_F01_si454.wav,Female\PTDB-TUG
2,Data/Audio/Female\PTDB-TUG\mic_F01_si473.wav,Female\PTDB-TUG
3,Data/Audio/Female\PTDB-TUG\mic_F01_si502.wav,Female\PTDB-TUG
4,Data/Audio/Female\PTDB-TUG\mic_F01_si523.wav,Female\PTDB-TUG


In [34]:
def load_data(spec_dir):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_generator = datagen.flow_from_directory(
        spec_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode="categorical",
        subset="training",
        shuffle=True
    )

    val_generator = datagen.flow_from_directory(
        spec_dir,
        target_size=(150, 150),
        batch_size=16,
        class_mode="categorical",
        subset="validation",
        shuffle=True
    )

    return train_generator, val_generator

In [35]:
def build_encoder(input_size):
    model = tf.keras.Sequential([
        Conv2D(32, (3, 3), input_shape=input_size, activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(2, 2),
        Dropout(0.2),

        Conv2D(64, (3, 3), activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(2, 2),
        Dropout(0.2),

        Conv2D(128, (3, 3), activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPool2D(2, 2),
        Dropout(0.2),

        GlobalMaxPool2D()
    ])
    return model
def build_model(input_size):
    input_layer = Input(input_size)
    encoder = build_encoder(input_size)(input_layer)
    output_layer = Dense(3, activation='softmax')(encoder)
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

def compile_model(model):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [36]:
def train_model(model, train_generator, val_generator):
    checkpoint_filepath = 'model/ckpt_noise/checkpoint.model.keras'
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True
    )

    early = EarlyStopping(
        patience=5,
        min_delta=0.001,
        restore_best_weights=True
    )

    model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=[early])

    return model
def evaluate_model(model, val_generator):
    return model.evaluate(val_generator)

# Load data
spec_dir = "noisy_spectrogram/"
train_generator, val_generator = load_data(spec_dir)

# Build model
input_size = train_generator.image_shape
model = build_model(input_size)

# Compile model
compiled_model = compile_model(model)

# Train model
trained_model = train_model(compiled_model, train_generator, val_generator)

# Evaluate model
evaluation_result = evaluate_model(trained_model, val_generator)
print("Evaluation Result:", evaluation_result)

Found 576 images belonging to 3 classes.
Found 143 images belonging to 3 classes.
Epoch 1/20
18/18 [==============================] - 32s 2s/step - loss: 1.0756 - accuracy: 0.4444 - val_loss: 1.0699 - val_accuracy: 0.4545
Epoch 2/20
18/18 [==============================] - 33s 2s/step - loss: 1.0751 - accuracy: 0.4514 - val_loss: 1.0734 - val_accuracy: 0.4545
Epoch 3/20
18/18 [==============================] - 33s 2s/step - loss: 1.0718 - accuracy: 0.4514 - val_loss: 1.0710 - val_accuracy: 0.4545
Epoch 4/20
18/18 [==============================] - 33s 2s/step - loss: 1.0615 - accuracy: 0.4514 - val_loss: 1.0619 - val_accuracy: 0.4545
Epoch 5/20
18/18 [==============================] - 33s 2s/step - loss: 1.0499 - accuracy: 0.4514 - val_loss: 1.0463 - val_accuracy: 0.4545
Epoch 6/20
18/18 [==============================] - 33s 2s/step - loss: 0.8416 - accuracy: 0.5451 - val_loss: 0.5102 - val_accuracy: 0.8601
Epoch 7/20
18/18 [==============================] - 33s 2s/step - loss: 0.6655

In [37]:
model.save('model.h6')

INFO:tensorflow:Assets written to: model.h6\assets


INFO:tensorflow:Assets written to: model.h6\assets


In [38]:
k=tf.keras.models.load_model('model.h6')
k.evaluate(val_generator)

9/9 [==============================] - 3s 240ms/step - loss: 0.0088 - accuracy: 1.0000


[0.00878223031759262, 1.0]